# Step 3: Changing % Column and Splitting out reports

Source: Eagle 'Reg 28 Report - Incl Effective Exposure' with nine columns: 0. 'Entity Name', 1. 'Investment Type', 2. 'i Issue Name', 3. 'Primary Asset ID', 4. 'CCY', 5. 'Reg28 Classification', 6. 'End Market Value', 7. 'Percentage', 8. 'Closing Exposure PA'

#### Resources

structures                       P:\Working Folders\Hilton\W\structures.xlsm

Reg28 sheet format and formula   P:\Working Folders\Hilton\W\Reg28 Checker Format and Formulae.docx

input file                       P:\Working Folders\Hilton\W\Reg 28 Report - input.xlsx

issuer results file              P:\Working Folders\Hilton\W\issuers_1_done.xlsx

merged file                      P:\Working Folders\Hilton\W\merged.xlsx

output file                      P:\Working Folders\Hilton\W\issuers_2_done.xlsx

In [1]:
# create a time difference function
import datetime, time

def timediff(start, end, decimals = 1):
    if   int((end - start) / 3600) > 0: # non-zero hours
        return str(int(  (end - start) / 3600))           + 'hr '  + \
               str(int(  (end - start) /   60))           + 'min ' + \
               str(round((end - start) %   60, decimals)) + 'sec'
    elif int((end - start) /   60) > 0: # non-zero hours and minutes
        return str(int(  (end - start) /   60))           + 'min ' + \
               str(round((end - start) %   60, decimals)) + 'sec'
    else:
        return str(round((end - start) %   60, decimals)) + 'sec'

In [2]:
# libraries, libraries!

    # time, to determine run durations
start_time          = time.time()
start_time_overlord = start_time

    # general libraries
import pandas   as     pd        # for dataframes
import numpy    as     np        # for np.NaN
import re                        # for regex
from   re       import search    # for regex
import os
from   datetime import datetime  # for script run durations

    # utilise openpyxl tools to add excel features to results sheet
import openpyxl        as     px                                          # for adding sort filters to the excel sheet
from   openpyxl.cell   import Cell                                        # to format cells
from   openpyxl.styles import Alignment, Color, PatternFill, Font, Border # to format cells

    # access Excel application to open results sheet
import win32com.client as win32                                           # library to convert xls to xlsx
excel = win32.gencache.EnsureDispatch('Excel.Application')                # to open excel application

from tqdm import tqdm, notebook # notebook version of tqdm
#" ...any time you see a loop somewhere in your code in you can simply wrap it in either tdqm() or tqdm_notebook() in Jupyter" 

print(timediff(start_time, time.time()))

# Possible error message when loading libraries, labraries!:
  # "TypeError: This COM object can not automate the makepy process - please run makepy manually for this object"
  # Solution: close all running Excel applications
  # https://stackoverflow.com/questions/57030278/typeerror-this-com-object-can-not-automate-the-makepy-process-please-run-make

1.0sec


In [3]:
# (2) get report date - https://stackoverflow.com/questions/43544514/pandas-read-specific-excel-cell-value-into-a-variable
pthW             = r'P:\Working Folders\Hilton\W'                   # path to working folder
input_filename   = r'\Reg 28 Report - input.xlsx'                   # input file name

k                = pd.read_excel(pthW + input_filename, index_col = None, usecols = 'K', nrows = 0)
report_date      = list(k)[0] # https://realpython.com/lessons/list-methods-return-values/

In [4]:
# set paths
start_time = time.time()

pthCompliance    = r'P:\Investment Operations\GRC\Compliance'       # path to PIM Compliance folder
pthW             = r'P:\Working Folders\Hilton\W'
pthBondData      = pthCompliance + r'\Reporting Requirements\Monthly Reports\MonthEnd17.xlsm'
input_filename   = r'\Reg 28 Report - input.xlsx'
results_filename = r'\issuers_2_done.xlsx'                          # output file name
pthReports       = r'P:\Investment Operations\GRC\Compliance\Reg28 and Reg30 Reporting'
output_folder    = r'P:\Working Folders\Hilton\W\Reg_Tests'

print(timediff(start_time, time.time()))

0.0sec


In [5]:
# change the 'Percentage of Market Value' column to the fund-specific % of NAV based on 'Sum of Market Value Income' column (M)
start_time = time.time()

#results    = pd.read_excel('P:\Working Folders\Hilton\W\issuers_2_done.xlsx', sheet_name = 'all')
results    = pd.read_excel(pthW + results_filename, sheet_name = 'all')
fund_codes = results['Entity Name'].unique()
navs       = results.groupby('Entity Name')['End Market Value'].sum() # (column N) this has type 'pandas.core.series.Series'
nav        = navs.to_dict() # nav series changed to dictionary to make it lookupable

# recalc the '% of Total Market Value' column per fund and ...
newTMV = []
for i, row in notebook.tqdm(results.iterrows()):
    if nav[row['Entity Name']] == 0:
        fndpct = 100
    else:
        fndpct = row['End Market Value'] / nav[row['Entity Name']] * 100
    newTMV.append(fndpct)
    
# ... replace 'Percentage of Market Value' with the values in the new list
results['Percentage of Market Value'] = newTMV
print('%.2f' % (results['Percentage of Market Value'].sum() / 100), len(fund_codes)) #check, should equal number of funds

print(timediff(start_time, time.time()))

0it [00:00, ?it/s]

1.00 1
0.2sec


In [6]:
# get a list of unique fund codes

colsR28    = ['Entity Name', 'Investment Type', 'i Issue Name', 'Primary Asset ID', 'CCY', 'Reg 28 Classification',
              'End Market Value', 'Percentage of Market Value', 'Closing Exposure PA', 'Issuer', 'Infra']
colsR30    = ['Entity Name', 'Investment Type', 'i Issue Name', 'Primary Asset ID', 'CCY', 'Reg 30 Classification',
              'End Market Value', 'Percentage of Market Value', 'Closing Exposure PA', 'Issuer']

R28 = results[colsR28]
R30 = results[colsR30]

In [7]:
# save the Reg 28 fund reports individually in the temporary resultant fodler

start_time = time.time()

pthReports = r'P:\Working Folders\Hilton\W\Reg_Tests'

for fund_code in fund_codes:
    fname = os.path.join(pthReports, f'{fund_code} Reg28 {report_date.strftime("%d%b%Y")}.xlsx')
    R28[R28['Entity Name'] == fund_code].to_excel(fname, sheet_name = f'{fund_code} Reg28 {report_date.strftime("%d%b%Y")}', index = False)
    
print(timediff(start_time, time.time()))

0.2sec


In [8]:
# save the Reg 30 fund reports individually

start_time = time.time()

for fund_code in fund_codes:
    fname = os.path.join(pthReports, f'{fund_code} Reg30 {report_date.strftime("%d%b%Y")}.xlsx')
    R30[R30['Entity Name'] == fund_code].to_excel(fname, sheet_name = f'{fund_code} Reg30 {report_date.strftime("%d%b%Y")}', index = False)
    
print(timediff(start_time, time.time()))
print(timediff(start_time_overlord, time.time()))

0.2sec
1.8sec


In [9]:
# open the resultant folder and the reporting folder

os.startfile(os.path.realpath(output_folder))
os.startfile(os.path.realpath(pthReports))

In [10]:
# dataframes:  example, no_issuer, no_CLN_issuer
# structures   P:\Working Folders\Hilton\W\structures.xlsm
# results      P:\Working Folders\Hilton\W\issuers_1_done.xlsx
# merged       P:\Working Folders\Hilton\W\merged.xlsx
# output       P:\Working Folders\Hilton\W\issuers_2_done.xlsx
# Reg 28 & 30  P:\Investment Operations\GRC\Compliance\Reg28 and Reg30 Reporting
# folder       P:\Working Folders\Hilton\W\Reg_Tests